This notebook generates measurement data with noise for orbit determination purposes, using a Keplerian propagator:

* Position/Velocity
* Range as seen from a list of ground stations
* Range rate (doppler) as seen from a list of ground stations
* Az/el as seen from a list of ground stations

The list of (example, not real) ground stations is loaded from the CSV file `ground-stations.csv`

Orbit parameters

In [1]:
import numpy as np
import datetime
orbit_epoch = datetime.datetime(2020, 1, 1)
sma = 10000e3  # Semi-major axis in meters
ecc = 0.2  # Eccentricity
inc = float(np.deg2rad(60.0))  # Inclination in radians
raan = 0.0  # Right-ascension of ascending node in radians
aop = 0.0  # Argument of perigee in radians
ta = 0.0  # Initial true anomaly in radians

n_orbits = 1.0  # Number of orbits to generate

orbit_period = 2*np.pi*np.sqrt(sma**3 / 3.98600e14)
duration = float(n_orbits*orbit_period)
display(duration)

9952.019565792982

Measurement parameters. For each type of measurement, two files are generated, one with the perfect measurements, one with noise added.
The noise for each type of measurement is modeled by a gaussian distribution with standard deviation $\sigma$

In [2]:
sigma_position = 100e3  # Position noise in meters
sigma_velocity = 100.0  # Velocity noise in meters per second
step_pv = 500.0  # Step time for output PV data

sigma_range = 1e3  # Range noise in meters
sigma_range_rate = 10e3  # Range rate noise in meters per second
sigma_az = float(np.deg2rad(1.0))  # Azimuth noise in radians
sigma_el = float(np.deg2rad(1.0))  # Elevation noise in radians
step_ground_station_measurements = 30.0  # When a ground station is in view, take measurements every 30 seconds

Loading the ground stations

In [3]:
import pandas as pd
ground_station_df = pd.read_csv('ground-stations.csv', index_col=0, )
display(ground_station_df)

,longitude_deg,latitude_deg,altitude
name,,,
Svalbard,15.0,78.0,0.0
Australia,117.0,-32.0,0.0
Argentina,-65.0,-35.0,0.0
Antarctica,-75.0,-73.0,0.0


Firing up a JVM for Orekit

In [4]:
import orekit
orekit.initVM()

Downloading and importing the Orekit data ZIP

In [5]:
from orekit.pyhelpers import download_orekit_data_curdir, setup_orekit_curdir
download_orekit_data_curdir()
setup_orekit_curdir()

Setting up models (frames, timescales)

In [6]:
from org.orekit.frames import FramesFactory
gcrf = FramesFactory.getGCRF()

from org.orekit.utils import IERSConventions
itrf = FramesFactory.getITRF(IERSConventions.IERS_2010, False)  # Taking tidal effects into account when interpolating EOP parameters

from org.orekit.models.earth import ReferenceEllipsoid
wgs84_ellipsoid = ReferenceEllipsoid.getWgs84(itrf)

from org.orekit.time import TimeScalesFactory
utc = TimeScalesFactory.getUTC()

Setting up initial orbit and propagator

In [7]:
from org.orekit.orbits import KeplerianOrbit, PositionAngle
from orekit.pyhelpers import datetime_to_absolutedate
from org.orekit.utils import Constants as orekit_constants
orbit = KeplerianOrbit(sma, ecc, inc, aop, raan, ta,
                       PositionAngle.TRUE, 
                       gcrf,
                       datetime_to_absolutedate(orbit_epoch),
                       orekit_constants.EIGEN5C_EARTH_MU)

from org.orekit.propagation.analytical import KeplerianPropagator
propagator = KeplerianPropagator(orbit)

Orekit can generate measurements for all types of measurements that are used for OD. Building the measurements builders:

Creating the Orekit ground station objects from DataFrame data.

Creating a measurement generator for each ground station and for each type of measurement

In [8]:
from org.orekit.bodies import GeodeticPoint
from org.orekit.frames import TopocentricFrame
from org.orekit.propagation.events import ElevationDetector
from org.orekit.estimation.measurements import GroundStation
from org.orekit.estimation.measurements.generation import PVBuilder, RangeBuilder, RangeRateBuilder, AngularAzElBuilder, EventBasedScheduler, SignSemantic, Generator, ContinuousScheduler
from org.orekit.time import FixedStepSelector
from org.hipparchus.random import JDKRandomGenerator, GaussianRandomGenerator, CorrelatedRandomVectorGenerator
from org.hipparchus.linear import DiagonalMatrix
from org.orekit.estimation.measurements import ObservableSatellite
gaussian_random_generator = GaussianRandomGenerator(JDKRandomGenerator())

# Noise generator for range
noise_source_range = CorrelatedRandomVectorGenerator(
    DiagonalMatrix([sigma_range**2]),
    1e-6,  # "small" parameter
    gaussian_random_generator
)

# Noise generator for range rate
noise_source_range_rate = CorrelatedRandomVectorGenerator(
    DiagonalMatrix([sigma_range_rate**2]),
    1e-6,  # "small" parameter
    gaussian_random_generator
)

# Noise generator for AZ/EL
noise_source_az_el = CorrelatedRandomVectorGenerator(
    DiagonalMatrix([sigma_az**2, sigma_el**2]),
    1e-6,  # "small" parameter
    gaussian_random_generator
)

# Noise generator for PV
noise_source_pv = CorrelatedRandomVectorGenerator(
    DiagonalMatrix([sigma_position**2, sigma_position**2, sigma_position**2, 
                    sigma_velocity**2, sigma_velocity**2, sigma_velocity**2]),
    1e-6,  # "small" parameter
    gaussian_random_generator
)

# Builder for PV measurements
observable_satellite = ObservableSatellite(0)
pv_builder = PVBuilder(noise_source_pv, 
                       sigma_position,
                       sigma_velocity,
                       1.0,  # Base weight
                       observable_satellite)

# Step selectors for ground station measurements
step_selector_range = FixedStepSelector(step_ground_station_measurements, utc)
step_selector_range_rate = FixedStepSelector(step_ground_station_measurements, utc)
step_selector_az_el = FixedStepSelector(step_ground_station_measurements, utc)

# Step selector for PV measurements
step_selector_pv = FixedStepSelector(step_pv, utc)
pv_scheduler = ContinuousScheduler(pv_builder, step_selector_pv)

meas_generator = Generator()
meas_generator.addPropagator(propagator)
meas_generator.addScheduler(pv_scheduler)

for gs_name, gs_data in ground_station_df.iterrows():
    geodetic_point = GeodeticPoint(float(np.deg2rad(gs_data['latitude_deg'])),
                                   float(np.deg2rad(gs_data['longitude_deg'])),
                                   float(gs_data['altitude']))
    topocentric_frame = TopocentricFrame(wgs84_ellipsoid, geodetic_point, gs_name)
    elevation_detector = ElevationDetector(topocentric_frame)
    #ground_station_df.loc[gs_name, 'ElevationDetector'] = elevation_detector 
    ground_station = GroundStation(topocentric_frame)
    ground_station_df.loc[gs_name, 'GroundStation'] = ground_station
    
    # Range builder
    range_builder = RangeBuilder(noise_source_range, 
                                 ground_station, 
                                 True,  # two-way
                                 sigma_range, 
                                 1.0,  # Base weight
                                 observable_satellite)
    #ground_station_df.loc[gs_name, 'RangeBuilder'] = range_builder
    range_scheduler = EventBasedScheduler(range_builder, 
                                          step_selector_range, 
                                          propagator, 
                                          elevation_detector, 
                                          SignSemantic.FEASIBLE_MEASUREMENT_WHEN_POSITIVE)
    meas_generator.addScheduler(range_scheduler)
    
    # Range rate builder
    range_rate_builder = RangeRateBuilder(noise_source_range_rate, 
                                          ground_station, 
                                          True,  # two-way
                                          sigma_range_rate, 
                                          1.0,  # Base weight
                                          observable_satellite)
    #ground_station_df.loc[gs_name, 'RangeRateBuilder'] = range_rate_builder
    range_rate_scheduler = EventBasedScheduler(range_rate_builder, 
                                               step_selector_range_rate, 
                                               propagator, 
                                               elevation_detector, 
                                               SignSemantic.FEASIBLE_MEASUREMENT_WHEN_POSITIVE)
    meas_generator.addScheduler(range_rate_scheduler)
    
    # Az/el builder
    az_el_builder = AngularAzElBuilder(noise_source_az_el, 
                                       ground_station,
                                       [sigma_az, sigma_el], 
                                       [1.0, 1.0],  # Base weight
                                       observable_satellite)
    #ground_station_df.loc[gs_name, 'AngularAzElBuilder'] = az_el_builder
    az_el_scheduler = EventBasedScheduler(az_el_builder, 
                                          step_selector_az_el, 
                                          propagator, 
                                          elevation_detector, 
                                          SignSemantic.FEASIBLE_MEASUREMENT_WHEN_POSITIVE)
    meas_generator.addScheduler(az_el_scheduler)

In [9]:
from orekit.pyhelpers import absolutedate_to_datetime
date_start = datetime_to_absolutedate(orbit_epoch)
date_end = date_start.shiftedBy(duration)
date_current = date_start
generated = meas_generator.generate(date_start, date_end)

In [14]:
az_el_with_noise_df

,GroundStation,az_deg,el_deg
2020-01-01 00:18:30,Svalbard,-86.69194638052997,1.1464687701674587
2020-01-01 00:19:00,Svalbard,-86.65159812004688,2.4200843292265573
2020-01-01 00:19:30,Svalbard,-88.01610636972592,5.596138330741703
2020-01-01 00:20:00,Svalbard,-89.88555920477047,5.800636611104817
2020-01-01 00:20:30,Svalbard,-90.14927835888594,5.631304435432498
...,...,...,...
2020-01-01 01:02:30,Svalbard,142.09537868244212,3.3494029772874483
2020-01-01 01:03:00,Svalbard,141.84395798995723,1.996042150264656
2020-01-01 01:03:30,Svalbard,143.58466225811958,0.8205902855086317
2020-01-01 01:04:00,Svalbard,142.2854986411128,-0.07412432678360537


In [10]:
from org.orekit.estimation.measurements import ObservedMeasurement, PV, Range, RangeRate, AngularAzEl
pv_with_noise_df = pd.DataFrame(columns=['x', 'y', 'z', 'vx', 'vy', 'vz'])
range_with_noise_df = pd.DataFrame(columns=['ground_station', 'range'])
range_rate_with_noise_df = pd.DataFrame(columns=['ground_station', 'range_rate'])
az_el_with_noise_df = pd.DataFrame(columns=['ground_station', 'az_deg', 'el_deg'])

for meas in generated.toArray():
    observed_meas = ObservedMeasurement.cast_(meas)
    py_datetime = absolutedate_to_datetime(observed_meas.date)
    
    if PV.instance_(observed_meas):
        observed_pv = PV.cast_(observed_meas)
        pv_with_noise_df.loc[py_datetime] = np.array(observed_meas.getObservedValue())
    elif Range.instance_(observed_meas):
        observed_range = Range.cast_(observed_meas)
        ground_station = observed_range.getStation()
        range_with_noise_df.loc[py_datetime] = [ground_station.getBaseFrame().name, observed_range.getObservedValue()]
    elif RangeRate.instance_(observed_meas):
        observed_range_rate = RangeRate.cast_(observed_meas)
        ground_station = observed_range_rate.getStation()
        range_rate_with_noise_df.loc[py_datetime] = [ground_station.getBaseFrame().name, observed_range_rate.getObservedValue()]
    elif AngularAzEl.instance_(observed_meas):
        observed_az_el = AngularAzEl.cast_(observed_meas)
        ground_station = observed_az_el.getStation()
        az_el = np.array(observed_az_el.getObservedValue())
        az_el_with_noise_df.loc[py_datetime] = np.concatenate(([ground_station.getBaseFrame().name], 
                                                                np.rad2deg(az_el)))

Finally, saving measurement data to CSV file

In [ ]:
position_df.to_csv('pos_vel_data_gcrf_with_noise.csv')
position_without_noise_df.to_csv('pos_vel_data_gcrf_without_noise.csv')